# Libor Curve Building Choice of Interpolation Scheme

We examine the impact of the choice of the interpolation scheme

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from financepy.utils import *
from financepy.products.rates import *
from financepy.market.curves import *

We start by defining this useful function

In [ ]:
def plotCurve(libor_curve, start_dt, tmax):
    years = np.linspace(1/365, tmax, 500)
    dates = start_dt.add_years(years)
    zero_rates = libor_curve.zero_rate(dates)
    fwd_rates = libor_curve.fwd(dates)

    plt.figure(figsize=(8,6))
    plt.plot(years, zero_rates*100, label="zero rates")
    plt.plot(years, fwd_rates*100, label = "fwd rates")
    plt.xlabel("Times")
    plt.ylabel("Rates (%) - See Legend")
    plt.legend()

## Deposits Only

We use deposits only for simplicity

### Piecewise Flat Interpolation

In [ ]:
spot_dt = Date(6, 6, 2018)

In [ ]:
spot_days = 0
settle_dt = spot_dt.add_weekdays(spot_days)
depoDCCType = DayCountTypes.ACT_360

In [ ]:
depos = []
depo = IborDeposit(settle_dt, "1M", 0.0230, depoDCCType); depos.append(depo)
depo = IborDeposit(settle_dt, "2M", 0.0235, depoDCCType); depos.append(depo)
depo = IborDeposit(settle_dt, "3M", 0.0240, depoDCCType); depos.append(depo)
depo = IborDeposit(settle_dt, "6M", 0.0220, depoDCCType); depos.append(depo)

libor_curve = IborSingleCurve(settle_dt, depos, [], [])

In [ ]:
print(libor_curve)

In [ ]:
plotCurve(libor_curve, settle_dt, 0.5)

The shape of the forwards is an artifact of the piecewise flat interpolation scheme and the fact that Actual 360 deposit rates do not compare directly to zero rates or continuously compounded forwards.

### Piecewise Linear Forwards Interpolation

In [ ]:
interpMethod = InterpTypes.LINEAR_FWD_RATES

depos = []
depo = IborDeposit(settle_dt, "1M", 0.0230, depoDCCType); depos.append(depo)
depo = IborDeposit(settle_dt, "2M", 0.0235, depoDCCType); depos.append(depo)
depo = IborDeposit(settle_dt, "3M", 0.0240, depoDCCType); depos.append(depo)
depo = IborDeposit(settle_dt, "6M", 0.0220, depoDCCType); depos.append(depo)

libor_curve = IborSingleCurve(settle_dt, depos, [], [], interpMethod)

plotCurve(libor_curve, settle_dt, 0.5)

### Piecewise Linear Zero Rates

In [ ]:
interpMethod = InterpTypes.LINEAR_ZERO_RATES

depos = []
depo = IborDeposit(settle_dt, "1M", 0.0230, depoDCCType); depos.append(depo)
depo = IborDeposit(settle_dt, "2M", 0.0235, depoDCCType); depos.append(depo)
depo = IborDeposit(settle_dt, "3M", 0.0245, depoDCCType); depos.append(depo)
depo = IborDeposit(settle_dt, "6M", 0.0220, depoDCCType); depos.append(depo)

libor_curve = IborSingleCurve(settle_dt, depos, [], [], interpMethod)

plotCurve(libor_curve, settle_dt, 0.5)

## Futures Only

Now we build a curve using just Futures contracts.

In [ ]:
spot_days = 2
settle_dt = spot_dt.add_weekdays(spot_days)

In [ ]:
futs = []
fut = IborFuture(spot_dt, 1) ; futs.append(fut)
fut = IborFuture(spot_dt, 2) ; futs.append(fut)
fut = IborFuture(spot_dt, 3) ; futs.append(fut)
fut = IborFuture(spot_dt, 4) ; futs.append(fut)
fut = IborFuture(spot_dt, 5) ; futs.append(fut)
fut = IborFuture(spot_dt, 6) ; futs.append(fut)

Given a futures price and a convexity adjustent, we can convert these to FRAs

In [ ]:
fras = [None]*len(futs)
fras[0] = futs[0].to_fra(97.6675,-0.00005)
fras[1] = futs[1].to_fra(97.5200,-0.00060)
fras[2] = futs[2].to_fra(97.3550,-0.00146)
fras[3] = futs[3].to_fra(97.2450,-0.00263)
fras[4] = futs[4].to_fra(97.1450,-0.00411)
fras[5] = futs[5].to_fra(97.0750,-0.00589)

In [ ]:
libor_curve = IborSingleCurve(settle_dt, [], fras, [])

plotCurve(libor_curve, settle_dt, 2)

We see that the forward rates are piecewise flat and increasing.

Copyright (c) 2020 Dominic O'Kane